# Анализ тенденций игровой индустрии в начале XXI века

- Автор: Чимбир В.И.
- Дата: 03.03.2025

### Цели и задачи проекта

Нормализовать данные, изучить развитие игровой индустрии с 2000 по 2013 год, анализируя оценки пользователей и критиков, а так же популярность платформ

### Описание данных

- `Name` — название игры.
- `Platform` — название платформы.
- `Year of Release` — год выпуска игры.
- `Genre` — жанр игры.
- `NA sales` — продажи в Северной Америке (в миллионах проданных копий).
- `EU sales` — продажи в Европе (в миллионах проданных копий).
- `JP sales` — продажи в Японии (в миллионах проданных копий).
- `Other sales` — продажи в других странах (в миллионах проданных копий).
- `Critic Score` — оценка критиков (от 0 до 100).
- `User Score` — оценка пользователей (от 0 до 10).
- `Rating` — рейтинг организации ESRB (англ. Entertainment Software Rating Board). Эта ассоциация определяет рейтинг компьютерных игр и присваивает им подходящую возрастную категорию.

### Содержимое проекта

1. Загрузка данных и знакомство с ними.
2. Проверка ошибок в данных и их предобработка.
3. Фильтрация данных
4. Категоризация данных
5. Итоговый вывод

---

##  Загрузка данных и знакомство с ними

- Загрузите необходимые библиотеки Python и данные датасета `/datasets/new_games.csv`.


In [1]:
# Подключаем библиотеку pandas
import pandas as pd
pd.options.mode.chained_assignment = None

In [2]:
# Загружаем датафрейм
df = pd.read_csv('https://code.s3.yandex.net/datasets/new_games.csv')



In [3]:
# Выводим первые пять строк датафрейма
df.head()

,Name,Platform,Year of Release,Genre,NA sales,EU sales,JP sales,Other sales,Critic Score,User Score,Rating
0,Wii Sports,Wii,2006.0,Sports,41.36,28.96,3.77,8.45,76.0,8,E
1,Super Mario Bros.,NES,1985.0,Platform,29.08,3.58,6.81,0.77,NaN,NaN,NaN
2,Mario Kart Wii,Wii,2008.0,Racing,15.68,12.76,3.79,3.29,82.0,8.3,E
3,Wii Sports Resort,Wii,2009.0,Sports,15.61,10.93,3.28,2.95,80.0,8,E
4,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,11.27,8.89,10.22,1.00,NaN,NaN,NaN


In [4]:
# Выводим информацию о датафрейме
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16956 entries, 0 to 16955
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Name             16954 non-null  object 
 1   Platform         16956 non-null  object 
 2   Year of Release  16681 non-null  float64
 3   Genre            16954 non-null  object 
 4   NA sales         16956 non-null  float64
 5   EU sales         16956 non-null  object 
 6   JP sales         16956 non-null  object 
 7   Other sales      16956 non-null  float64
 8   Critic Score     8242 non-null   float64
 9   User Score       10152 non-null  object 
 10  Rating           10085 non-null  object 
dtypes: float64(4), object(7)
memory usage: 1.4+ MB


-----

Промежуточные выводы о датафрейме:

- Следует привести все столбцы к единому виду snake case
- Следует изменить тип данных некоторых стобцов (`EU sales`,`JP sales`,`User Score` с `object` на `float64`, `Year of Release` и `Critic Score` к `int64`)
- Наблюдаем большое количество пропусков в некоторыз столбцах. Следует сделать преобразование.

---

##  Проверка ошибок в данных и их предобработка


###  Названия, или метки, столбцов датафрейма



In [5]:
# Выводим на экран названия столбцов датафрейма
df.columns

Index(['Name', 'Platform', 'Year of Release', 'Genre', 'NA sales', 'EU sales',
       'JP sales', 'Other sales', 'Critic Score', 'User Score', 'Rating'],
      dtype='object')

In [6]:
# Редактируем стиль написания столбцов датафрейма
df = df.rename(columns={
                         'Name':'name',
                         'Platform': 'platform',
                         'Year of Release': 'year_of_release',
                         'Genre': 'genre',
                         'NA sales': 'na_sales',
                         'EU sales': 'eu_sales',
                         'JP sales': 'jp_sales',
                         'Other sales': 'other_sales',
                         'Critic Score': 'critic_score',
                         'User Score': 'user_score',
                         'Rating': 'rating'
})

In [7]:
# Проверяем результат редактирования
df.columns

Index(['name', 'platform', 'year_of_release', 'genre', 'na_sales', 'eu_sales',
       'jp_sales', 'other_sales', 'critic_score', 'user_score', 'rating'],
      dtype='object')

### Типы данных



In [47]:
# Проверяем уникальные значения стобцов с целью выявления некорректных данных
df['eu_sales'].unique()

array([2.89600000e+01, 3.58000000e+00, 1.27600000e+01, 1.09300000e+01,
       8.89000000e+00, 2.26000000e+00, 9.14000000e+00, 9.18000000e+00,
       6.94000000e+00, 6.30000000e-01, 1.09500000e+01, 7.47000000e+00,
       6.18000000e+00, 8.03000000e+00, 4.89000000e+00, 8.49000000e+00,
       9.09000000e+00, 4.00000000e-01, 3.75000000e+00, 9.20000000e+00,
       4.46000000e+00, 2.71000000e+00, 3.44000000e+00, 5.14000000e+00,
       5.49000000e+00, 3.90000000e+00, 5.35000000e+00, 3.17000000e+00,
       5.09000000e+00, 4.24000000e+00, 5.04000000e+00, 5.86000000e+00,
       3.68000000e+00, 4.19000000e+00, 5.73000000e+00, 3.59000000e+00,
       4.51000000e+00, 2.55000000e+00, 4.02000000e+00, 4.37000000e+00,
       6.31000000e+00, 3.45000000e+00, 2.81000000e+00, 2.85000000e+00,
       3.49000000e+00, 1.00000000e-02, 3.35000000e+00, 2.04000000e+00,
       3.07000000e+00, 3.87000000e+00, 3.00000000e+00, 4.82000000e+00,
       3.64000000e+00, 2.15000000e+00, 3.69000000e+00, 2.65000000e+00,
      

In [9]:
df['jp_sales'].unique()

array(['3.77', '6.81', '3.79', '3.28', '10.22', '4.22', '6.5', '2.93',
       '4.7', '0.28', '1.93', '4.13', '7.2', '3.6', '0.24', '2.53',
       '0.98', '0.41', '3.54', '4.16', '6.04', '4.18', '3.84', '0.06',
       '0.47', '5.38', '5.32', '5.65', '1.87', '0.13', '3.12', '0.36',
       '0.11', '4.35', '0.65', '0.07', '0.08', '0.49', '0.3', '2.66',
       '2.69', '0.48', '0.38', '5.33', '1.91', '3.96', '3.1', '1.1',
       '1.2', '0.14', '2.54', '2.14', '0.81', '2.12', '0.44', '3.15',
       '1.25', '0.04', '0.0', '2.47', '2.23', '1.69', '0.01', '3.0',
       '0.02', '4.39', '1.98', '0.1', '3.81', '0.05', '2.49', '1.58',
       '3.14', '2.73', '0.66', '0.22', '3.63', '1.45', '1.31', '2.43',
       '0.7', '0.35', '1.4', '0.6', '2.26', '1.42', '1.28', '1.39',
       '0.87', '0.17', '0.94', '0.19', '0.21', '1.6', '0.16', '1.03',
       '0.25', '2.06', '1.49', '1.29', '0.09', '2.87', '0.03', '0.78',
       '0.83', '2.33', '2.02', '1.36', '1.81', '1.97', '0.91', '0.99',
       '0.95', '2.0'

In [10]:
df['user_score'].unique()

array(['8', nan, '8.3', '8.5', '6.6', '8.4', '8.6', '7.7', '6.3', '7.4',
       '8.2', '9', '7.9', '8.1', '8.7', '7.1', '3.4', '5.3', '4.8', '3.2',
       '8.9', '6.4', '7.8', '7.5', '2.6', '7.2', '9.2', '7', '7.3', '4.3',
       '7.6', '5.7', '5', '9.1', '6.5', 'tbd', '8.8', '6.9', '9.4', '6.8',
       '6.1', '6.7', '5.4', '4', '4.9', '4.5', '9.3', '6.2', '4.2', '6',
       '3.7', '4.1', '5.8', '5.6', '5.5', '4.4', '4.6', '5.9', '3.9',
       '3.1', '2.9', '5.2', '3.3', '4.7', '5.1', '3.5', '2.5', '1.9', '3',
       '2.7', '2.2', '2', '9.5', '2.1', '3.6', '2.8', '1.8', '3.8', '0',
       '1.6', '9.6', '2.4', '1.7', '1.1', '0.3', '1.5', '0.7', '1.2',
       '2.3', '0.5', '1.3', '0.2', '0.6', '1.4', '0.9', '1', '9.7'],
      dtype=object)

In [11]:
# Преобразовывваем типы данных в нужных столбцах на числовые, меняя строчные значения на пропуск 
for columns in ['eu_sales','jp_sales','user_score']:
    df[columns] = pd.to_numeric(df[columns],errors = 'coerce')

In [12]:
# Проверяем результаты преобразования
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16956 entries, 0 to 16955
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   name             16954 non-null  object 
 1   platform         16956 non-null  object 
 2   year_of_release  16681 non-null  float64
 3   genre            16954 non-null  object 
 4   na_sales         16956 non-null  float64
 5   eu_sales         16950 non-null  float64
 6   jp_sales         16952 non-null  float64
 7   other_sales      16956 non-null  float64
 8   critic_score     8242 non-null   float64
 9   user_score       7688 non-null   float64
 10  rating           10085 non-null  object 
dtypes: float64(7), object(4)
memory usage: 1.4+ MB


Для дальнешей корректной работы с датафреймом были сделаны следующие изменения:
- столбцы `eu_sales`,`jp_sales` и `user_score` типа object преобразовали в тип float64.


### Наличие пропусков в данных




In [13]:
# Выводим количество пропущенных строк в датафрейме
df.isna().sum()

name                  2
platform              0
year_of_release     275
genre                 2
na_sales              0
eu_sales              6
jp_sales              4
other_sales           0
critic_score       8714
user_score         9268
rating             6871
dtype: int64

In [14]:
# Подсчитаем процент строк с пропусками
df.isna().sum() / len(df) * 100

name                0.011795
platform            0.000000
year_of_release     1.621845
genre               0.011795
na_sales            0.000000
eu_sales            0.035386
jp_sales            0.023590
other_sales         0.000000
critic_score       51.391838
user_score         54.659118
rating             40.522529
dtype: float64

In [15]:
df.isna().sum() / len(df) * 100

name                0.011795
platform            0.000000
year_of_release     1.621845
genre               0.011795
na_sales            0.000000
eu_sales            0.035386
jp_sales            0.023590
other_sales         0.000000
critic_score       51.391838
user_score         54.659118
rating             40.522529
dtype: float64

-------

Изучаем пропуски:

- Наблюдаем очень незначительный процент (от ~0,02% до ~0,035%) пропусков в столбцах `eu_sales` и `jp_sales`. Их можно заполнить средним значением по году выхода игры и названию платформы.  

- Больше трети данных (40%) отсутсвует в столбе `rating`. Так как эти данные в дальнейшем анализе нам не пригодятся - заменим значение на `unknown`

- Больше чем у половины игр (51% и 54% соответственно) отсутсвуют значения в столбцах `user_score` и `critic_score`. Как показала фильтрация - пропуски в этих столбцах никак не зависят от времени. Удаление данных или замена на среднее значение - сильно исказят аналитическую картину, поэтому заменим пропуски значением-индикатором.
- ~1,62% пропусков в столбце `year_of_release`. Так как столбец содержит информацию о годе выпуска - заполнить его какими-либо приближенными показателями невозможно. Поэтому используем так же значение индикатор.
- Совсем незначильные пропуски (0,01%) в столбцах `name` и `genre` и они содержат строковые данные, поэтому заполним их строкой `unknown`

--------

In [16]:
# Заполняем пропуски в eu_sales
## Создаем функцию для вычисления среднего знакчения по категориям

def mean_group_eu_sales(row):
    if pd.isna(row['eu_sales']):
        group = df[(df['year_of_release'] == row['year_of_release']) & 
        (df['platform'] == row['platform'])]
        return group['eu_sales'].mean()
    else:
        return row['eu_sales']

### Применяем функцию на серию

df['eu_sales'] =  df.apply(mean_group_eu_sales,axis=1)



In [17]:
# Заполняем пропуски в jp_sales
## Создаем функцию для вычисления среднего знакчения по категориям

def mean_group_jp_sales(row):
    if pd.isna(row['jp_sales']):
        group = df[(df['year_of_release'] == row['year_of_release']) & 
        (df['platform'] == row['platform'])]
        return group['jp_sales'].mean()
    else:
        return row['jp_sales']

### Применяем функцию на серию

df['jp_sales'] =  df.apply(mean_group_jp_sales,axis=1)



In [18]:
# Заполняем данные в столбцах 'name', 'genre' и 'rating' 

df['name'] = df['name'].fillna('unknown')
df['genre'] = df['name'].fillna('unknown')
df['rating'] = df['rating'].fillna('unknown')

In [19]:
# Проверяем уникальные значения перед заполнением пропусков
df['year_of_release'].unique()

array([2006., 1985., 2008., 2009., 1996., 1989., 1984., 2005., 1999.,
       2007., 2010., 2013., 2004., 1990., 1988., 2002., 2001., 2011.,
       1998., 2015., 2012., 2014., 1992., 1997., 1993., 1994., 1982.,
       2016., 2003., 1986., 2000.,   nan, 1995., 1991., 1981., 1987.,
       1980., 1983.])

In [20]:
df['user_score'].unique()

array([8. , nan, 8.3, 8.5, 6.6, 8.4, 8.6, 7.7, 6.3, 7.4, 8.2, 9. , 7.9,
       8.1, 8.7, 7.1, 3.4, 5.3, 4.8, 3.2, 8.9, 6.4, 7.8, 7.5, 2.6, 7.2,
       9.2, 7. , 7.3, 4.3, 7.6, 5.7, 5. , 9.1, 6.5, 8.8, 6.9, 9.4, 6.8,
       6.1, 6.7, 5.4, 4. , 4.9, 4.5, 9.3, 6.2, 4.2, 6. , 3.7, 4.1, 5.8,
       5.6, 5.5, 4.4, 4.6, 5.9, 3.9, 3.1, 2.9, 5.2, 3.3, 4.7, 5.1, 3.5,
       2.5, 1.9, 3. , 2.7, 2.2, 2. , 9.5, 2.1, 3.6, 2.8, 1.8, 3.8, 0. ,
       1.6, 9.6, 2.4, 1.7, 1.1, 0.3, 1.5, 0.7, 1.2, 2.3, 0.5, 1.3, 0.2,
       0.6, 1.4, 0.9, 1. , 9.7])

In [21]:
df['critic_score'].unique()

array([76., nan, 82., 80., 89., 58., 87., 91., 61., 97., 95., 77., 88.,
       83., 94., 93., 85., 86., 98., 96., 90., 84., 73., 74., 78., 92.,
       71., 72., 68., 62., 49., 67., 81., 66., 56., 79., 70., 59., 64.,
       75., 60., 63., 69., 50., 25., 42., 44., 55., 48., 57., 29., 47.,
       65., 54., 20., 53., 37., 38., 33., 52., 30., 32., 43., 45., 51.,
       40., 46., 39., 34., 35., 41., 36., 28., 31., 27., 26., 19., 23.,
       24., 21., 17., 22., 13.])

In [22]:
# Заполняем пропуски индикатором "-1" в столбцах critic_score и user_score

df['user_score'] = df['user_score'].fillna(-1)
df['critic_score'] = df['critic_score'].fillna(-1)
df['year_of_release'] = df['year_of_release'].fillna(-1)

In [23]:
# Теперь привидем столбцы 'year_of_release' и 'critic_score' к целочисленному типу данных
df['year_of_release'] = df['year_of_release'].astype('int64')
df['critic_score'] = df['critic_score'].astype('int64')

In [24]:
# Проверим результат преобразования
df[['critic_score','year_of_release']]

,critic_score,year_of_release
0,76,2006
1,-1,1985
2,82,2008
3,80,2009
4,-1,1996
...,...,...
16951,-1,2016
16952,-1,2006
16953,-1,2016
16954,-1,2003


In [25]:
# Проверяем результат
df.isna().sum() / len(df) * 100

name               0.0
platform           0.0
year_of_release    0.0
genre              0.0
na_sales           0.0
eu_sales           0.0
jp_sales           0.0
other_sales        0.0
critic_score       0.0
user_score         0.0
rating             0.0
dtype: float64

###  Явные и неявные дубликаты в данных



In [26]:
# Редактируем регистры, с целью минимизировать неявные дубликаты

df['name'] = df['name'].str.lower() 
df['genre'] = df['genre'].str.lower() 
df['platform'] = df['platform'].str.lower() 
df['rating'] = df['rating'].str.upper() 

In [27]:
# Проверяем уникальные значения в столбце 'genre'
df.sort_values(by='genre')['genre'].unique()

array([' beyblade burst', ' fire emblem fates', " frozen: olaf's quest",
       ..., 'zwei!!', 'zyuden sentai kyoryuger: game de gaburincho!!',
       '¡shin chan flipa en colores!'], dtype=object)

In [28]:
# Проверяем уникальные значения в столбце 'year_of_release'
df.sort_values(by='year_of_release')['year_of_release'].unique()

array([  -1, 1980, 1981, 1982, 1983, 1984, 1985, 1986, 1987, 1988, 1989,
       1990, 1991, 1992, 1993, 1994, 1995, 1996, 1997, 1998, 1999, 2000,
       2001, 2002, 2003, 2004, 2005, 2006, 2007, 2008, 2009, 2010, 2011,
       2012, 2013, 2014, 2015, 2016])

In [29]:
# Проверяем уникальные значения в столбце 'rating'
df.sort_values(by='rating')['rating'].unique()

array(['AO', 'E', 'E10+', 'EC', 'K-A', 'M', 'RP', 'T', 'UNKNOWN'],
      dtype=object)

In [30]:
# Проверяем уникальные значения в столбце 'platform'
df.sort_values(by='platform')['platform'].unique()

array(['2600', '3do', '3ds', 'dc', 'ds', 'gb', 'gba', 'gc', 'gen', 'gg',
       'n64', 'nes', 'ng', 'pc', 'pcfx', 'ps', 'ps2', 'ps3', 'ps4', 'psp',
       'psv', 'sat', 'scd', 'snes', 'tg16', 'wii', 'wiiu', 'ws', 'x360',
       'xb', 'xone'], dtype=object)

Проделанная работа по поиску по неявных дубликатов:

- Нормализовали регистры для каждого стобца.
- Проверили уникальне значения категориальных данных на опечатки предарительно отсортировав данные в порядке возрастания

Итоги: неявных дубликатов не обнаружено.

In [31]:
# Проверяем общее кол-во строк датафрейма
df.shape[0]

16956

In [32]:
# Узнаем сумму явных дубликатов
df.duplicated().sum()

241

In [33]:
# Высчитаем процент дубликатов от датафрема
df.duplicated().sum() / df.shape[0] * 100

1.421325784383109

In [34]:
# Удаляем дубликаты
df = df.drop_duplicates()

In [35]:
# Проверяем результат
df.duplicated().sum()

0

In [36]:
df.shape[0]

16715

Проделанная работа по явным дубликатам:

- Найдено и удалено 241 дубликатов, что составляет приблизительно 1,4% от нашего датафрейма

-------

Итоги и промежуточные выводы предобработки данных:

- Все даннные преобразованы в необходимый для дальнейшего анализа тип
- Часть пропусков имели очень маленький процент относительно кол-ва строк(меньше процента) поэтому были успешно заполненны усредненными данными, которые не исказят аналитическую картину. Однако другая часть данных имеет слишком большой процент пропусков(больше 30%). Было принято решение заменить их значением - индикатором. 
- Проведена успешая работа с неявнымми и явными дубликатами, доля которых составила лишь 1,4% от датафрейма.
- Датафрейрм отфильтрован в связи с потребнотями поставленных задач. Все данные успешно нормализованы, за исключением некоторызз пропусков.

---

##  Фильтрация данных

Коллеги хотят изучить историю продаж игр в начале XXI века, и их интересует период с 2000 по 2013 год включительно. Отберите данные по этому показателю. Сохраните новый срез данных в отдельном датафрейме, например `df_actual`.

In [37]:
# Отфильтруем данные во временном интервале, необходимом для задачи
df_actual = df[(df['year_of_release'] >= 2000) & (df['year_of_release'] <= 2013)]

In [38]:
df_actual.head()

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating
0,wii sports,wii,2006,wii sports,41.36,28.96,3.77,8.45,76,8.0,E
2,mario kart wii,wii,2008,mario kart wii,15.68,12.76,3.79,3.29,82,8.3,E
3,wii sports resort,wii,2009,wii sports resort,15.61,10.93,3.28,2.95,80,8.0,E
6,new super mario bros.,ds,2006,new super mario bros.,11.28,9.14,6.50,2.88,89,8.5,E
7,wii play,wii,2006,wii play,13.96,9.18,2.93,2.84,58,6.6,E


---

## Категоризация данных
    
Проведите категоризацию данных:
- Разделите все игры по оценкам пользователей и выделите такие категории: высокая оценка (от 8 до 10 включительно), средняя оценка (от 3 до 8, не включая правую границу интервала) и низкая оценка (от 0 до 3, не включая правую границу интервала).

In [39]:
# Разделяем данные на категории по столбцы 'user_score'
df_actual['user_score_group'] = pd.cut(df_actual['user_score'],bins=[-1.1,0,2,7,10], labels=["без оценки","низкая оценка","средняя оценка","высокая оценка"])

In [40]:
df_actual.head()

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating,user_score_group
0,wii sports,wii,2006,wii sports,41.36,28.96,3.77,8.45,76,8.0,E,высокая оценка
2,mario kart wii,wii,2008,mario kart wii,15.68,12.76,3.79,3.29,82,8.3,E,высокая оценка
3,wii sports resort,wii,2009,wii sports resort,15.61,10.93,3.28,2.95,80,8.0,E,высокая оценка
6,new super mario bros.,ds,2006,new super mario bros.,11.28,9.14,6.50,2.88,89,8.5,E,высокая оценка
7,wii play,wii,2006,wii play,13.96,9.18,2.93,2.84,58,6.6,E,средняя оценка


- Разделите все игры по оценкам критиков и выделите такие категории: высокая оценка (от 80 до 100 включительно), средняя оценка (от 30 до 80, не включая правую границу интервала) и низкая оценка (от 0 до 30, не включая правую границу интервала).

In [41]:
# Разделяем данные на категории по столбцу 'critic_score'
df_actual['critic_score_group'] = pd.cut(df_actual['critic_score'],bins=[-1.1,0,29,79,100], labels=["без оценки","низкая оценка","средняя оценка","высокая оценка"])

In [42]:
df_actual.head()

,name,platform,year_of_release,genre,na_sales,eu_sales,jp_sales,other_sales,critic_score,user_score,rating,user_score_group,critic_score_group
0,wii sports,wii,2006,wii sports,41.36,28.96,3.77,8.45,76,8.0,E,высокая оценка,средняя оценка
2,mario kart wii,wii,2008,mario kart wii,15.68,12.76,3.79,3.29,82,8.3,E,высокая оценка,высокая оценка
3,wii sports resort,wii,2009,wii sports resort,15.61,10.93,3.28,2.95,80,8.0,E,высокая оценка,высокая оценка
6,new super mario bros.,ds,2006,new super mario bros.,11.28,9.14,6.50,2.88,89,8.5,E,высокая оценка,высокая оценка
7,wii play,wii,2006,wii play,13.96,9.18,2.93,2.84,58,6.6,E,средняя оценка,средняя оценка


- После категоризации данных проверьте результат: сгруппируйте данные по выделенным категориям и посчитайте количество игр в каждой категории.

In [43]:
# Группируем данные по новым категориям и вычисляем кол-во игр в кажой категории
df_actual.groupby('user_score_group')['user_score_group'].count()

user_score_group
без оценки        6299
низкая оценка       46
средняя оценка    2351
высокая оценка    4085
Name: user_score_group, dtype: int64

In [44]:
df_actual.groupby('critic_score_group')['critic_score_group'].count()

critic_score_group
без оценки        5612
низкая оценка       55
средняя оценка    5422
высокая оценка    1692
Name: critic_score_group, dtype: int64

- Выделите топ-7 платформ по количеству игр, выпущенных за весь актуальный период.

In [45]:
# Выводим топ-7 платформ 
df_actual.groupby('platform')['platform'].count().sort_values(ascending=False).head(7)

platform
ps2     2127
ds      2120
wii     1275
psp     1180
x360    1121
ps3     1087
gba      811
Name: platform, dtype: int64

---

## Итоговый вывод



- Сделана полная предобратка данных (преобразование стиля, изменеие типов данных, редатирование пропусков, удаление дубликатов)
- Проведена нобходимая для задачи фильтрация и категоризация данных

In [46]:
# Выгрузим информацию о финальном варианте датафрейма в нужном срезе и с новыми категориями
df_actual.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 12781 entries, 0 to 16954
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype   
---  ------              --------------  -----   
 0   name                12781 non-null  object  
 1   platform            12781 non-null  object  
 2   year_of_release     12781 non-null  int64   
 3   genre               12781 non-null  object  
 4   na_sales            12781 non-null  float64 
 5   eu_sales            12781 non-null  float64 
 6   jp_sales            12781 non-null  float64 
 7   other_sales         12781 non-null  float64 
 8   critic_score        12781 non-null  int64   
 9   user_score          12781 non-null  float64 
 10  rating              12781 non-null  object  
 11  user_score_group    12781 non-null  category
 12  critic_score_group  12781 non-null  category
dtypes: category(2), float64(5), int64(2), object(4)
memory usage: 1.2+ MB


Обновленное описание данных:

- `name` — название игры.
- `platform` — название платформы.
- `year_of_release` — год выпуска игры.
- `genre` — жанр игры.
- `na_sales` — продажи в Северной Америке (в миллионах проданных копий).
- `eu_sales` — продажи в Европе (в миллионах проданных копий).
- `jp_sales` — продажи в Японии (в миллионах проданных копий).
- `other_sales` — продажи в других странах (в миллионах проданных копий).
- `critic_score` — оценка критиков (от 0 до 100).
- `user_score` — оценка пользователей (от 0 до 10).
- `rating` — рейтинг организации ESRB (англ. Entertainment Software Rating Board). Эта ассоциация определяет рейтинг компьютерных игр и присваивает им подходящую возрастную категорию.
- `user_score_group` - категория в зависимости от оценки пользователя (без оценки, высркая,средняя,низкая).
- `critic_score_group` - - категория в зависимости от оценки критиков (без оценки, высркая,средняя,низкая).

Выводы:

- В топе платформ по релизу игр наблюдаем двух явных лидеров: `ps2` и `ds` опережающих следующую группу платформ приблизительно на 900 релизов
- Преобладаемая категория в срезах оценок критиков и пользователей - "без оценки". Что сильно ослажняет анализ ввиду простого отсутсвия данных. Однако наблюаем одинковую тенденцию - в обоих срезах самая значительно непопулярная категория (55 и 46 против ближайших показателей 1692 и 2351 соотвестввенно) - "низкая оценка". Так же наблюдаем, что пользоваетели более лояльне чем критики, ведь второй по поулярности результат у первых - "высокая оценка" в отличии от "средняя оценка" вторых.